# Creating a very simple dynamic bioMod file
For the forward dynamics part, using a seven degrees of freedom model will be much to complicated.
So the purpose of this extra file is to create a very simple model of one degree of freedom. 
For that, we are stripping all the lower part of the full body model we have been using up to now. 

You will therefore find a lot of similarities between this file and the `2-dynamic_model_creation` file, with the only exception that everything below the pelvis is removed.

WARNING: Don't forget to run the `1-inverse_kinematics` file with `use_upper_limb` set to `True` after running the current script so data for upper limb can be used later on!

In [ ]:
# Input paths
kinematic_model_file_path = "models/SimpleBody.bioMod"
kinematic_chain_file_path = "models/simple_body_kinematic_chain.kc"

# Output paths
dynamic_model_file_path = "models/SimpleUpperLimbWithInertia.bioMod"
kinematic_chain_save_file_path = "models/upper_limb_kinematic_chain_with_inertia.kc"

## Reload the kinematic model
In order ot load the kinematic chain (saved using pickle), we must define the classes that were used. 
So the next cell is a copy of the class declaration of `0-kinematic_model_creation`.

In [ ]:
# First of all, let's make our life easier by defining a class that we will fill for each of the segment
# Notice that mass, center_of_mass and inertia_xxyyzz are currently ignored. They will become relevant in 
# dynamics section below
class Segment():
    def __init__(
        self, 
        name, 
        parent_name="", 
        rt="",
        translations="", 
        rotations="",
        mass=0,
        center_of_mass=None,
        inertia_xxyyzz=None,
        mesh: tuple=None,
        markers: tuple=None,
    ):
        self.name = name
        self.parent_name = parent_name
        self.rt = rt
        self.translations = translations
        self.rotations = rotations
        self.mass = mass
        self.center_of_mass=center_of_mass
        self.inertia_xxyyzz=inertia_xxyyzz
        self.mesh = mesh
        self.markers = markers

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"segment {self.name}\n"
        if (self.parent_name):
            out_string += f"\tparent {self.parent_name}\n"
        if (self.rt):
            out_string += f"\tRT {self.rt}\n"
        if (self.translations):
            out_string += f"\ttranslations {self.translations}\n"
        if (self.rotations):
            out_string += f"\trotations {self.rotations}\n"
        if (self.mass):
            out_string += f"\tmass {self.mass}\n"
        if (self.center_of_mass):
            out_string += f"\tcom {self.center_of_mass[0]} {self.center_of_mass[1]} {self.center_of_mass[2]}\n"
        if (self.inertia_xxyyzz):
            out_string += f"\tinertia {self.inertia_xxyyzz[0]} 0 0\n" + \
                          f"\t        0 {self.inertia_xxyyzz[1]} 0\n" + \
                          f"\t        0 0 {self.inertia_xxyyzz[2]}\n"
        if (self.mesh):
            for m in self.mesh:
                out_string += f"\tmesh {m[0]} {m[1]} {m[2]}\n"
        out_string += "endsegment\n"
        
        # Also print the markers attached to the segment
        if (self.markers):
            for marker in self.markers:
                out_string += str(marker)
        return out_string
    
    
# Let's do the same for the markers
class Marker():
    def __init__(
        self, 
        name, 
        parent_name, 
        position,
    ):
        self.name = name
        self.parent_name = parent_name
        self.position = position

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"marker {self.name}\n"
        out_string += f"\tparent {self.parent_name}\n"
        out_string += f"\tposition {self.position[0]} {self.position[1]} {self.position[2]}\n"
        out_string += "endmarker\n"
        return out_string


# And why not creating a data structure that will collect all the segments and produce the a full
# kinematic chain that can easily be printed
class KinematicChain():
    def __init__(self, segments):
        self.segments = segments
        
    def __str__(self):
        out_string = "version 4\n\n"
        for segment in self.segments:
            out_string += str(segment)
            out_string += "\n\n\n"  # Give some space between segments
        return out_string
    
    def write(self, file_path):
        # Method to write the current KinematicChain to a file
        with open(file_path, "w") as file:
            file.write(str(self))

In [ ]:
# So let's first reload the kinematic chain created in `0-kinematic_model_creation` (saved using pickle)
# Meaning that after running this cell, the `kinematic_chain` variable is exactly what it was at the end of `0-kinematic_model_creation` with the lower body part removed
import pickle

with open(kinematic_chain_file_path, "rb") as file:
    kinematic_chain = pickle.load(file)

# Collect the individual segments in the kinematic chain
trunk = kinematic_chain.segments[0]
head = kinematic_chain.segments[1]
upper_arm = kinematic_chain.segments[2]
lower_arm = kinematic_chain.segments[3]
hand = kinematic_chain.segments[4]

# Recreate the kinematic chain using only upper part
kinematic_chain = KinematicChain(segments=(trunk, head, upper_arm, lower_arm, hand))


## Extend to dynamic model
For that second part, we will add the inertia parameters to the model. 
It will allow us, for instance, to perform inverse dynamics analyses.

We are using the De Leva inertia parameter model.

In [ ]:
# Now let's use the De Leva table to produce realistic inertia value for our model
# We are using biorbd (the bioMod) file to make some computation. Please note that it is
# by no way mandatory, and we could have used the KinematicChain class we created. But
# I thought it was a good occasion to introduce how to retreive some information from biorbd
import numpy

# Let's first copy the De Leva table in a dictionary using a simple DeLeva class
class DeLeva:
    # This is an internal class just to make things well sorted
    class Param:    
        def __init__(
            self,
            marker_names,  # The name of the markers medial/lateral
            mass,  # Percentage of the total body mass
            center_of_mass,  # Position of the center of mass as a percentage of the distance from medial to distal
            radii,  # [Sagittal, Transverse, Longitudinal] radii of giration
        ):
            self.marker_names = marker_names
            self.mass = mass
            self.center_of_mass = center_of_mass
            self.radii = radii

    def __init__(self, sex, mass, model):
        self.sex = sex  # The sex of the subject
        self.mass = mass  # The mass of the subject
        self.model = model  # The biorbd model. This is to compute lengths
        
        # Produce some easy to access variables
        self.q_zero = numpy.zeros((model.nbQ()))
        self.marker_names = [name.to_string() for name in model.markerNames()]
        
        # This is the actual copy of the De Leva table
        self.table = {}
        self.table["male"] = {}
        self.table["male"]["HEAD"]      = DeLeva.Param(marker_names=("TOP_HEAD", "SHOULDER"), mass=0.0694  , center_of_mass=0.5002, radii=(0.303, 0.315, 0.261))
        self.table["male"]["TRUNK"]     = DeLeva.Param(marker_names=("SHOULDER", "PELVIS")  , mass=0.4346  , center_of_mass=0.5138, radii=(0.328, 0.306, 0.169))
        self.table["male"]["UPPER_ARM"] = DeLeva.Param(marker_names=("SHOULDER", "ELBOW")   , mass=0.0271*2, center_of_mass=0.5772, radii=(0.285, 0.269, 0.158))
        self.table["male"]["LOWER_ARM"] = DeLeva.Param(marker_names=("ELBOW", "WRIST")      , mass=0.0162*2, center_of_mass=0.4574, radii=(0.276, 0.265, 0.121))
        self.table["male"]["HAND"]      = DeLeva.Param(marker_names=("WRIST", "FINGER")     , mass=0.0061*2, center_of_mass=0.7900, radii=(0.628, 0.513, 0.401))
        self.table["male"]["THIGH"]     = DeLeva.Param(marker_names=("PELVIS", "KNEE")      , mass=0.1416*2, center_of_mass=0.4095, radii=(0.329, 0.329, 0.149))
        self.table["male"]["SHANK"]     = DeLeva.Param(marker_names=("KNEE", "ANKLE")       , mass=0.0433*2, center_of_mass=0.4459, radii=(0.255, 0.249, 0.103))
        self.table["male"]["FOOT"]      = DeLeva.Param(marker_names=("ANKLE", "TOE")        , mass=0.0137*2, center_of_mass=0.4415, radii=(0.257, 0.245, 0.124))

        self.table["female"] = {}
        self.table["female"]["HEAD"]      = DeLeva.Param(marker_names=("TOP_HEAD", "SHOULDER"), mass=0.0669  , center_of_mass=0.4841, radii=(0.271, 0.295, 0.261))
        self.table["female"]["TRUNK"]     = DeLeva.Param(marker_names=("SHOULDER", "PELVIS")  , mass=0.4257  , center_of_mass=0.4964, radii=(0.307, 0.292, 0.147))
        self.table["female"]["UPPER_ARM"] = DeLeva.Param(marker_names=("SHOULDER", "ELBOW")   , mass=0.0255*2, center_of_mass=0.5754, radii=(0.278, 0.260, 0.148))
        self.table["female"]["LOWER_ARM"] = DeLeva.Param(marker_names=("ELBOW", "WRIST")      , mass=0.0138*2, center_of_mass=0.4559, radii=(0.261, 0.257, 0.094))
        self.table["female"]["HAND"]      = DeLeva.Param(marker_names=("WRIST", "FINGER")     , mass=0.0056*2, center_of_mass=0.7474, radii=(0.531, 0.454, 0.335))
        self.table["female"]["THIGH"]     = DeLeva.Param(marker_names=("PELVIS", "KNEE")      , mass=0.1478*2, center_of_mass=0.3612, radii=(0.369, 0.364, 0.162))
        self.table["female"]["SHANK"]     = DeLeva.Param(marker_names=("KNEE", "ANKLE")       , mass=0.0481*2, center_of_mass=0.4416, radii=(0.271, 0.267, 0.093))
        self.table["female"]["FOOT"]      = DeLeva.Param(marker_names=("ANKLE", "TOE")        , mass=0.0129*2, center_of_mass=0.4014, radii=(0.299, 0.279, 0.124))

    def segment_mass(self, segment):
        return self.table[self.sex][segment].mass * self.mass
    
    def segment_length(self, segment):
        table = self.table[self.sex][segment]
        
        # Find the position of the markers when the model is in resting position
        marker_positions = numpy.array([marker.to_array() for marker in self.model.markers(self.q_zero)]).transpose()
        
        # Find the index of the markers required to compute the length of the segment
        idx_proximal = self.marker_names.index(table.marker_names[0])
        idx_distal = self.marker_names.index(table.marker_names[1])
        
        # Compute the Euclidian distance between the two positions
        return numpy.linalg.norm(marker_positions[:, idx_distal] - marker_positions[:, idx_proximal])
        
    def segment_center_of_mass(self, segment, inverse_proximal=False):
        # This method will compute the length of the required segment based on the biorbd model and required markers
        # If inverse_proximal is set to True, then the value is returned from the distal position
        table = self.table[self.sex][segment]
        
        # Find the position of the markers when the model is in resting position
        marker_positions = numpy.array([marker.to_array() for marker in self.model.markers(self.q_zero)]).transpose()
        
        # Find the index of the markers required to compute the length of the segment
        idx_proximal = self.marker_names.index(table.marker_names[0])
        idx_distal = self.marker_names.index(table.marker_names[1])
        
        # Compute the position of the center of mass
        if inverse_proximal:
            center_of_mass = (1-table.center_of_mass) * (marker_positions[:, idx_proximal] - marker_positions[:, idx_distal])
        else:
            center_of_mass = table.center_of_mass * (marker_positions[:, idx_distal] - marker_positions[:, idx_proximal])
        return tuple(center_of_mass)  # convert the result to a Tuple which is good practise
        
    def segment_moment_of_inertia(self, segment):
        mass = self.segment_mass(segment)
        length = self.segment_length(segment)
        radii = self.table[self.sex][segment].radii
        
        return (mass * (length*radii[0])**2, mass * (length*radii[1])**2, mass * (length*radii[2])**2)

In [ ]:
# Let's suppose we have a male subject of 70kg which dimension are given by the bioMod previously created
deleva = DeLeva(sex="male", mass=70, model=biorbd.Model(kinematic_model_file_path))


# So let's fill all the field in each segments (and again let's define a convenient function that will do that for us)
def fill_segment(segment, deleva, inverse_proximal=False):
    segment.mass = deleva.segment_mass(segment.name)
    segment.center_of_mass = deleva.segment_center_of_mass(segment.name, inverse_proximal)
    segment.inertia_xxyyzz = deleva.segment_moment_of_inertia(segment.name)


# Fill all the segments
fill_segment(trunk, deleva, True)
fill_segment(head, deleva, True)
fill_segment(upper_arm, deleva)
fill_segment(lower_arm, deleva)
fill_segment(hand, deleva)

# Remove the translations and rotations at the trunk
trunk.translations = None
trunk.rotations = None

# Print and rewrite the bioMod file
print(kinematic_chain)
kinematic_chain.write(dynamic_model_file_path)

# As we did for the kinematic chain, let's save this kinematic chain so it can be reloaded as is later
with open(kinematic_chain_save_file_path, "wb") as file:
    pickle.dump(kinematic_chain, file)

In [ ]:
#
# THIS CELL IS COMMENTED AS IT CANNOT BE RUN FROM THE BINDER
# If you are running this notebook locally, you can uncomment the cell to visualize the model you just created
# 

# # Run this cell to show the model you just created. 
# # You should notice that the centers of mass now appear

# import bioviz

# # Send the previously loaded model to the vizualizer
# viz = bioviz.Viz(dynamic_model_file_path)

# # Move the model to a recognizable position (arm raised and knee flexed)
# viz.set_q((1.20,))

# # Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
# viz.exec()

# # If nothing happens, check for background. Sometimes the window loads behind the current window